# TensorFlow Recommenders: Quickstart

In this tutorial, we build a simple matrix factorization model using the [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/100k/) with TFRS. We can use this model to recommend movies for a given user.

# Import TensorFlow Recommender System (TFRS)

In [ ]:
!pip install -q tensorflow==2.3.0
!pip install -q tensorflow-recommenders==0.1.3
!pip install -q tensorflow-datasets==4.0.1

# Annoy
https://github.com/spotify/annoy

In [ ]:
!pip install -q annoy==1.16.3

In [ ]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

# Load Ratings

In [ ]:
# /home/ec2-user/tensorflow_datasets/movielens/100k-ratings/0.1.0/

ratings = tfds.load('movielens/100k-ratings', split='train')
print(ratings)

# Load Movies

In [ ]:
movies = tfds.load('movielens/100k-movies', split='train')
print(movies)

# Select Basic Features

In [ ]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
print(ratings)

In [ ]:
movies = movies.map(lambda x: x["movie_title"])
print(movies)

In [ ]:
#print(type(ratings))
#print(type(movies))

# Create Vocabularies
Build vocabularies to convert user ids and movie titles into integer indices for embedding layers:

In [ ]:
user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

In [ ]:
#print(user_ids_vocabulary.get_vocabulary())

# Create the Model

We can define a TFRS model by inheriting from `tfrs.Model` and implementing the `compute_loss` method:

In [ ]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

#    https://github.com/tensorflow/tensorflow/issues/31057
#     @tf.function(input_signature=[tf.TensorSpec([1], tf.float32)])
#     def call(self, x, training=True, mask=None):
#         return self.d(x)
    

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

# Define User and Movie Models
Define the two models.

In [ ]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 128)
])

movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 128)
])

# Define the Retrieval Task

In [ ]:
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)

# Train the Retrieval Model

In [ ]:
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train.
model.fit(ratings.batch(4096), epochs=10)

# Save TensorFlow Model

In [ ]:
import os

local_model_dir = './exported_models/'

tensorflow_saved_model_path = os.path.join(local_model_dir, 'tensorflow/saved_model/0')

os.makedirs(tensorflow_saved_model_path, exist_ok=True)

In [ ]:
# https://github.com/tensorflow/tensorflow/issues/31057

In [ ]:
# model.build([1])

In [ ]:
model.save(tensorflow_saved_model_path, save_format='tf')

# Make Predictions
Use brute-force search to set up retrieval using the trained representations.

In [ ]:
index = tfrs.layers.ann.BruteForce(model.user_model)
index.index(movies.batch(100).map(model.movie_model), movies)

In [ ]:
# Get some recommendations.
_, titles = index(np.array(["42"]))

print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

# Note we must call index() above before we save.  
# See https://github.com/tensorflow/tensorflow/issues/31057 for more details.

In [ ]:
index.save(tensorflow_saved_model_path, save_format='tf')

In [ ]:
!saved_model_cli show --all --dir $tensorflow_saved_model_path

In [40]:
restored_model = index.load(tensorflow_saved_model_path)
print(restored_model)

AttributeError: 'BruteForce' object has no attribute 'load'